<a href="https://colab.research.google.com/github/yyw-informatics/WASP/blob/main/Immcantation_1_VDJ_Annotation_and_Standardization_with_Change_O_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perform V(D)J annotation and data standardization

* Input data:     
  two files from the 10x genomics V(D)J protocol
  * `filtered_contigs_annotations.csv` (sequence annotations)
  * `filtered_contigs.fasta` (sequences for alignment)
* Tasks: 
  * annotate contigs with V(D)J genes 
  * produce a standard file in the format of Adaptive Immune Receptor Repertoire (AIRR)
* Output data:
  * `data_db-pass.tsv`

### Install Python packages: 

#### Presto and Changeo

In [1]:
!pip3 install presto changeo --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 4.2 MB/s 
     |████████████████████████████████| 150 kB 38.4 MB/s 
     |████████████████████████████████| 2.3 MB 42.1 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
  Created wheel for presto: filename=presto-0.7.0-py3-none-any.whl size=118994 sha256=926508d1da7122344ed22d5799387d572c5182be670dbcf8b3ba42895b478189
  Stored in directory: /root/.cache/pip/wheels/6d/0a/a3/e14b65e7f79b61fe7041df3ccb89d362291c21802cdda52dfa
  Created wheel for changeo: filename=changeo-1.2.0-py3-none-any.whl size=151660 sha256=4be154052979d541917a2bd878073d2e228478b9149d7f6f106dbeb8927e6b40
  Stored in directory: /root/.cache/pip/wheels/5b/4c/bf/cc895c5f294dd2e6fd20546fa6dbecb8814448b2ad65d4613b
  Created wheel for airr: filename=airr-1.3.1-py3-none-any.whl size=33408 sha256=f5a45d7164c588f

#### Add to $PATH

In [2]:
import os
!echo $PATH
os.environ['PATH'] += ":/root/.local/bin"
os.environ['PATH'] += ":/root/.local/lib/python3.7/site-packages"
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/root/.local/bin:/root/.local/lib/python3.7/site-packages


### Git clone the immcantation repo from bitbucket


1.   Login in [bitbucket](https://bitbucket.org/dashboard/overview)
2.   Go to repo [Immcantation](https://bitbucket.org/kleinstein/immcantation/src/master/)
3.   Click on the button `Clone` to copy the following path
4.   Folder `immcantation` will show under `Files`


In [3]:
!git clone https://Anna_2021@bitbucket.org/kleinstein/immcantation.git

Cloning into 'immcantation'...
remote: Enumerating objects: 5030, done.
remote: Counting objects: 100% (5030/5030), done.
remote: Compressing objects: 100% (2781/2781), done.
remote: Total 5030 (delta 3282), reused 3485 (delta 2201), pack-reused 0
Receiving objects: 100% (5030/5030), 8.08 MiB | 6.91 MiB/s, done.
Resolving deltas: 100% (3282/3282), done.


#### Add to $PATH variable


In [4]:
!echo $PATH
os.environ['PATH'] += ":/content/immcantation/scripts"
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/root/.local/bin:/root/.local/lib/python3.7/site-packages
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/root/.local/bin:/root/.local/lib/python3.7/site-packages:/content/immcantation/scripts


### IgBLAST and IMGT references

#### Download IgBLAST
#### Check the most recent version here
https://ftp.ncbi.nih.gov/blast/executables/igblast/release/LATEST/

In [5]:
VERSION='1.19.0'
!wget https://ftp.ncbi.nih.gov/blast/executables/igblast/release/LATEST/ncbi-igblast-$VERSION-x64-linux.tar.gz
!tar -zxf ncbi-igblast-$VERSION-x64-linux.tar.gz
!rm -r ncbi-igblast-$VERSION-x64-linux.tar.gz

--2022-08-02 14:19:41--  https://ftp.ncbi.nih.gov/blast/executables/igblast/release/LATEST/ncbi-igblast-1.19.0-x64-linux.tar.gz
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.12, 130.14.250.10, 2607:f220:41f:250::229, ...
Connecting to ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54748923 (52M) [application/x-gzip]
Saving to: ‘ncbi-igblast-1.19.0-x64-linux.tar.gz’

ncbi-igblast-1.19.0 100%[===================>]  52.21M  14.7MB/s    in 4.2s    

2022-08-02 14:19:47 (12.3 MB/s) - ‘ncbi-igblast-1.19.0-x64-linux.tar.gz’ saved [54748923/54748923]



#### Install IgBLAST

In [6]:
!mkdir -p /usr/local/bin/igblast
!cp -u ncbi-igblast-$VERSION/bin/* /usr/local/bin/igblast
!ls /usr/local/bin/igblast 

blastdbcmd  edit_imgt_file.pl  igblastn  igblastp  makeblastdb


#### Download references

In [7]:
!fetch_igblastdb.sh -o /content/drive/MyDrive/Immcantation/igblast
!cp -r ncbi-igblast-$VERSION/internal_data /content/drive/MyDrive/Immcantation/igblast
!cp -r ncbi-igblast-$VERSION/optional_file /content/drive/MyDrive/Immcantation/igblast

#### Add to $PATH variable

In [8]:
!echo $PATH
os.environ['PATH'] += ":/usr/local/bin/igblast"
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/root/.local/bin:/root/.local/lib/python3.7/site-packages:/content/immcantation/scripts
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/root/.local/bin:/root/.local/lib/python3.7/site-packages:/content/immcantation/scripts:/usr/local/bin/igblast


#### Build IgBLAST db from IMGT references

In [9]:
!fetch_imgtdb.sh -o /content/drive/MyDrive/Immcantation/germlines/igmt
!imgt2igblast.sh -i /content/drive/MyDrive/Immcantation/germlines/igmt -o /content/drive/MyDrive/Immcantation/igblast

|- VDJ regions
|---- Ig
|---- TCR
|- Spliced leader regions
|---- Ig
|---- TCR
|- Spliced constant regions
|---- Ig
|---- TCR

|- VDJ regions
|---- Ig
|---- TCR
|- Spliced leader regions
|---- Ig
|---- TCR
|- Spliced constant regions
|---- Ig
|---- TCR

|- VDJ regions
|---- Ig
|---- TCR
|- Spliced leader regions
|---- Ig
|---- TCR
|- Spliced constant regions
|---- Ig
|---- TCR

|- VDJ regions
|---- Ig
|---- TCR
|- Spliced leader regions
|---- Ig
|---- TCR
|- Spliced constant regions
|---- Ig
|---- TCR

|- VDJ regions
|---- Ig
|---- TCR
|- Spliced leader regions
|---- Ig
|---- TCR
|- Spliced constant regions
|---- Ig
|---- TCR



Building a new DB, current time: 08/02/2022 14:33:05
New DB name:   /content/drive/MyDrive/Immcantation/igblast/database/imgt_human_ig_c
New DB title:  /content/drive/MyDrive/Immcantation/igblast/fasta/imgt_human_ig_c.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 62 sequences in 0.097693 seconds.

### Locations


#### Python functions and pakcages


In [10]:
! ls /root/.local/bin

airr-tools	   BuildTrees.py       DefineClones.py	 ParseDb.py
AlignRecords.py    ClusterSets.py      EstimateError.py  ParseHeaders.py
AlignSets.py	   CollapseSeq.py      FilterSeq.py	 ParseLog.py
AssemblePairs.py   ConvertDb.py        MakeDb.py	 __pycache__
AssignGenes.py	   ConvertHeaders.py   MaskPrimers.py	 SplitSeq.py
BuildConsensus.py  CreateGermlines.py  PairSeq.py	 UnifyHeaders.py


In [11]:
!ls /root/.local/lib/python3.7/site-packages/

airr			  presto-0.7.0.dist-info
airr-1.3.1.dist-info	  __pycache__
Bio			  PyYAML-6.0.dist-info
biopython-1.79.dist-info  tests
BioSQL			  yaml
changeo			  _yaml
changeo-1.2.0.dist-info   yamlordereddictloader-0.4.0.dist-info
presto			  yamlordereddictloader.py


#### Additional scripts and pipelines
These are functions not included in the packages

In [12]:
!ls /content/immcantation/scripts

changeo2vdjtools.R  fetch_imgtdb.sh	light_cluster.py  run_igblast.sh
clean_imgtdb.py     fetch_phix.sh	merge10x.py
fastq2fasta.py	    imgt2cellranger.py	merge_fastq.py
fetch_igblastdb.sh  imgt2igblast.sh	mixcr2imgt.py


In [ ]:
!ls /content/immcantation/pipelines/

changeo-10x.sh	    presto-assemble.sh	    presto-index.sh
changeo-clone.sh    presto-clontech.sh	    shazam-threshold.R
changeo-igblast.sh  presto-clontech-umi.sh  tigger-genotype.R
preprocess-phix.sh  presto-consensus.sh
presto-abseq.sh     presto-filter.sh


#### Reference V(D)J germline genes from IMGT

In [13]:
!ls /content/drive/MyDrive/Immcantation/germlines/igmt

human  IMGT.yaml  mouse  rabbit  rat  rhesus_monkey


In [14]:
!ls /content/drive/MyDrive/Immcantation/igblast

database  fasta  internal_data	optional_file


## Mount Drive

### Remove previous mounted drive

In [26]:
!rm -rf drive/

### Mount drive

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Check access to input data
The first line should show folders
The second line should show first a few lines of the fasta file

In [28]:
!ls /content/drive/MyDrive/Immcantation/data

Rita_0622  vdj_0654  vdj_0654_subsampled  vdj_1140  vdj_1485  vdj_1490


In [29]:
!head /content/drive/MyDrive/Immcantation/data/vdj_0654/filtered_contig.fasta

>AAACCTGAGAAGGTTT-1_contig_1
AGGAGTCAGACCCAGTCAGGACACAGCATGGACATGAGGGTCCCCGCTCAGCTCCTGGGGCTCCTGCTGCTCTGGCTCCCAGGTGCCAGATGTGTCATCTGGATGACCCAGTCTCCATCCTTACTCTCTGCATCTACAGGAGACAGAGTCACCATCAGTTGTCGGATGAGTCAGGGCATTAGCAGTTATTTAGCCTGGTATCAGCAAAAACCAGGGAAAGCCCCTGAGCTCCTGATCTATGCTGCATCCACTTTGCAAAGTGGGGTCCCATCAAGGTTCAGTGGCAGTGGATCTGGGACAGATTTCACTCTCACCATCAGTTGCCTGCAGTCTGAAGATTTTGCAACTTATTACTGTCAACAGTATTATAGTTTCCCGTGGACGTTCGGCCAAGGGACCAAGGTGGAAATCAAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC
>AAACCTGAGAAGGTTT-1_contig_2
AGCTCTGAGAGAGGAGCCCAGCCCTGGGATTTTCAGGTGTTTTCATTTGGTGATCAGGACTGAACAGAGAGAACTCACCATGGAGTTTGGGCTGAGCTGGCTTTTTCTTGTGGCTATTTTAAAAGGTGTCCAGTGTGAGGTGCAGCTGTTGGAGTCTGGGGGAGGCTTGGTACAGCCTGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTAGCAGCTATGCCATGAGCTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTCTCAGCTATTAGTGGTAGTGGTGGTAGCACATACTACGCAGACTCCGTGAAGGGCCGGTTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAATGAACAG

## 1. V(D)J gene annotation

Use IgBLAST to annotate each read with its germline V(D)J gene alleles and to identify relevant sequence structure.

[`Change-O`](https://pubmed.ncbi.nlm.nih.gov/26069265/) provides a wrapper script `AssignGenes.py` to run IgBLAST.

[`AssignGenes.py`](https://changeo.readthedocs.io/en/stable/tools/AssignGenes.html) performs V(D)J assignment with IgBLAST.
* `-s`: input sequences  
* `-b`: a reference germlines database
* `--organism`: species 
* `--loci`: type of receptor, ig for BCR, tcr for TCR
* `--format blast`: **output file in the fmt7 format.**
* `--nproc`: the number of processors with .

In [ ]:
SAM='/vdj_0654/'
DIR='/content/drive/MyDrive/Immcantation/'
INPUT=DIR + 'data' + SAM + 'filtered_contig.fasta'
OUTPUT=DIR + 'results' + SAM
IGBLAST=DIR + 'igblast'
!mkdir -p $OUTPUT

In [ ]:
!AssignGenes.py igblast \
-s $INPUT \
-b $IGBLAST \
--organism human \
--loci ig \
--format blast \
--outdir $OUTPUT \
--nproc 8

   START> AssignGenes
 COMMAND> igblast
 VERSION> 1.19.0
    FILE> filtered_contig.fasta
ORGANISM> human
    LOCI> ig
   NPROC> 8

PROGRESS> 22:25:19 |Done                     | 12.5 min

  PASS> 17586
OUTPUT> filtered_contig_igblast.fmt7
   END> AssignGenes



## 2. Data standardization

The fmt7 results from previous step `IgBLAST` are converted into a standardized format, [AIRR](https://docs.airr-community.org/en/latest/datarep/rearrangements.html.) format, a tabulated text file with rows are sequences, and columns are annotations. 

[`MakeDb.py`](https://changeo.readthedocs.io/en/stable/tools/MakeDb.html) converts annotated resulst to AIRR format
* `-s`: input sequences
* **`--10x`:** the filtered_contig_annotations.csv file from 10x
* `-i`: annotated sequences
* `-r`: reference germlines
* `--format`: AIRR
* `--extend`: adds these columns:  `<vdj>_score`, `<vdj>_identity`, `<vdj>_support`, `<vdj>_cigar`, `fwr1`, `fwr2`, `fwr3`, `fwr4`, `cdr1`, `cdr2` and `cdr3`.

In [ ]:
CHANGEO=DIR + 'results' + SAM + 'changeo'
REF_VDJ=DIR +'germlines/igmt/human/vdj'
INPUT2=DIR + 'data' + SAM + 'filtered_contig_annotations.csv'
!mkdir -p $CHANGEO
!MakeDb.py igblast \
-s $INPUT \
--10x $INPUT2 \
-i $OUTPUT/filtered_contig_igblast.fmt7 \
-r $REF_VDJ \
--format airr \
--outdir $CHANGEO \
--outname data \
--extend  

         START> MakeDB
       COMMAND> igblast
  ALIGNER_FILE> filtered_contig_igblast.fmt7
      SEQ_FILE> filtered_contig.fasta
       ASIS_ID> False
    ASIS_CALLS> False
       PARTIAL> False
      EXTENDED> True
INFER_JUNCTION> False

PROGRESS> 22:30:24 |Done                | 0.0 min

PROGRESS> 22:30:47 |####################| 100% (17,586) 0.4 min

OUTPUT> data_db-pass.tsv
  PASS> 17496
  FAIL> 90
   END> MakeDb



Output file is:

In [ ]:
CHANGEO + '/data_db-pass.tsv'

'/content/drive/MyDrive/Immcantation/results/vdj_0654/changeo/data_db-pass.tsv'

Check the results file - preview the first row and all columns

In [ ]:
import pandas as pd 

In [ ]:
d = pd.read_csv(CHANGEO + '/data_db-pass.tsv', sep='\t', header=0)
d.iloc[0]

sequence_id                                 AAACCTGAGAAGGTTT-1_contig_1
sequence              AGGAGTCAGACCCAGTCAGGACACAGCATGGACATGAGGGTCCCCG...
rev_comp                                                              F
productive                                                            T
v_call                                                      IGKV1D-8*01
d_call                                                              NaN
j_call                                                         IGKJ1*01
sequence_alignment    GTCATCTGGATGACCCAGTCTCCATCCTTACTCTCTGCATCTACAG...
germline_alignment    GTCATCTGGATGACCCAGTCTCCATCCTTACTCTCTGCATCTACAG...
junction                              TGTCAACAGTATTATAGTTTCCCGTGGACGTTC
junction_aa                                                 CQQYYSFPWTF
v_cigar                                                         93S284=
d_cigar                                                             NaN
j_cigar                                                         

# Additional functions 

## Subset productive chains 

[`ParseDb.py`](https://changeo.readthedocs.io/en/stable/tools/ParseDb.html) to filter 'Productive' sequences. 

* `-d`: the input file
* `-f`: the columns to filer
* `-u`: the values in those columns to match
* `--regex`: treat values as regular expressions and allow partial string matches

In [ ]:
DATA=CHANGEO + '/data_db-pass.tsv'
!ParseDb.py select \
-d $DATA \
-f productive \
-u T \
--outname data_productive

  START> ParseDb
COMMAND> select
   FILE> data_db-pass.tsv
 FIELDS> productive
 VALUES> T
  REGEX> False

PROGRESS> 20:56:36 |####################| 100% (17,496) 0.0 min

   OUTPUT> data_productive_parse-select.tsv
  RECORDS> 17496
 SELECTED> 17496
DISCARDED> 0
      END> ParseDb



In [ ]:
dp = pd.read_csv(CHANGEO + '/data_productive_parse-select.tsv', sep='\t', header=0)
dp.iloc[0]

sequence_id                                 AAACCTGAGAAGGTTT-1_contig_1
sequence              AGGAGTCAGACCCAGTCAGGACACAGCATGGACATGAGGGTCCCCG...
rev_comp                                                              F
productive                                                            T
v_call                                                      IGKV1D-8*01
d_call                                                              NaN
j_call                                                         IGKJ1*01
sequence_alignment    GTCATCTGGATGACCCAGTCTCCATCCTTACTCTCTGCATCTACAG...
germline_alignment    GTCATCTGGATGACCCAGTCTCCATCCTTACTCTCTGCATCTACAG...
junction                              TGTCAACAGTATTATAGTTTCCCGTGGACGTTC
junction_aa                                                 CQQYYSFPWTF
v_cigar                                                         93S284=
d_cigar                                                             NaN
j_cigar                                                         

## Split heavy and light chain files

In [ ]:
DATA=CHANGEO + '/data_productive_parse-select.tsv'
!ParseDb.py select \
-d $DATA \
-f locus -u "IGH" \
--logic all \
--regex \
--outname heavy

  START> ParseDb
COMMAND> select
   FILE> data_productive_parse-select.tsv
 FIELDS> locus
 VALUES> IGH
  REGEX> True

PROGRESS> 21:01:30 |####################| 100% (17,496) 0.0 min

   OUTPUT> heavy_parse-select.tsv
  RECORDS> 17496
 SELECTED> 8477
DISCARDED> 9019
      END> ParseDb



In [ ]:
!ParseDb.py select \
-d $DATA \
-f locus \
-u "IG[LK]" \
--logic all \
--regex \
--outname light

  START> ParseDb
COMMAND> select
   FILE> data_productive_parse-select.tsv
 FIELDS> locus
 VALUES> IG[LK]
  REGEX> True

PROGRESS> 21:01:44 |####################| 100% (17,496) 0.0 min

   OUTPUT> light_parse-select.tsv
  RECORDS> 17496
 SELECTED> 9019
DISCARDED> 8477
      END> ParseDb



## One-step processing

In [ ]:
DIR='/content/drive/MyDrive/Immcantation/'
FASTA=DIR + 'data' + SAM + 'filtered_contig.fasta'
CSV=DIR + 'data' + SAM + 'filtered_contig_annotations.csv'
OUTPUT=DIR + 'results' + SAM + 'changeo-10x'

In [ ]:
!/content/immcantation/pipelines/changeo-10x.sh \
-s $FASTA \
-a $CSV \
-o $OUTPUT \
-g human \
-t ig \
-x auto